In [1]:
pwd()

"/Users/oldmtnbiker/Library/CloudStorage/OneDrive-Personal/evotech/CGP.jl/src"

In [2]:
using LinearAlgebra

In [3]:
include("CGP.jl")

UInt16

In [4]:
include("evolvable_evolvability.jl")

evolvability_list_filtered (generic function with 2 methods)

In [7]:
include("InfPop.jl")

mutation (generic function with 1 method)

In [8]:
p = Parameters(3,1,8,4);funcs=default_funcs(p)[1:4]

4-element Vector{Func}:
 Func(&, 2, "AND")
 Func(|, 2, "OR")
 Func(Main.CGP.Nand, 2, "NAND")
 Func(Main.CGP.Nor, 2, "NOR")

In [9]:
ddf = read_dataframe("../data/3_14_23/phnet_matrix3_14_23D.csv")   # A dataframe for sampling approx matrix

,goal,d_evolvability,s_evolvability,t_evolvability,0x0,0x1,0x2
,String7,Int64,Int64,Int64,Int64,Int64,Int64
1,0x0,245,2386637,247,8448871,46321,44614
2,0x1,206,188369,211,46277,191384,4455
3,0x2,198,130874,211,44158,4464,124506
4,0x3,178,250704,186,48454,18144,10842
5,0x4,193,99828,198,34146,3922,916
6,0x5,180,314932,193,69480,19434,262
7,0x6,100,1641,121,205,21,189
8,0x7,139,33601,156,3400,797,562
9,0x8,201,89539,212,30183,636,1251


In [11]:
E=df_to_matrix_mt( ddf, 5 )

256×256 Matrix{Int64}:
 8448871   46321   44614   48461  34312  …   23945    4705    7345   415301
   46277  191384    4455   18294   3764        988    2030   12480     7442
   44158    4464  124506   10939    876       1117    7999    1904     4747
   48454   18144   10842  314826    182      28185    1101     938    23595
   34146    3922     916     175  84612        544     176    1252     3315
   69480   19434     262    8524   8410  …      46     668     679    27708
     205      21     189      19    176          9      18      57       48
    3400     797     562    3085    357         97     471     144     2120
   30183     636    1251     218    841         90     143      47     2531
      12      40       2       2      3          0       4       2        6
   46617     158    6296     499    170  …       5      79     105     9694
     412     312     115     703     89         19      18      85      242
   24228      87     121    2647   3941       1997      97      7

In [12]:
P=fill(1.0,2^2^p.numinputs); P=P/sum(P); P'  # Initial uniform population

1×256 adjoint(::Vector{Float64}) with eltype Float64:
 0.00390625  0.00390625  0.00390625  …  0.00390625  0.00390625  0.00390625

In [17]:
labels = map(x->[x],MyInt(0):MyInt(2^2^p.numinputs-1));

In [43]:
(E*P)[1:10] # 1 generations of mutation with no selection
# Note that 0x05 and 0x0a have highest frequency even though these are not the phenotypes with highest redundany

10-element Vector{Float64}:
 42326.203125
  1483.41015625
   997.578125
  2209.1015625
   720.46875
  2842.734375
     8.83203125
   228.484375
   649.23828125
     1.17578125

In [44]:
(E*P)[0x0069:0x0073] 

11-element Vector{Float64}:
   0.27734375
   0.00390625
  28.21875
   0.23828125
  10.30078125
   0.05859375
  19.3203125
   1.19921875
 173.03125
   3.69140625
 127.4921875

In [45]:
 normalize(E*P)[0x0069:0x0072]  # 4 generations of mutation with no selection, normalized

10-element Vector{Float64}:
 1.42e-6
 2.0e-8
 0.00014448
 1.22e-6
 5.274e-5
 3.0e-7
 9.892e-5
 6.14e-6
 0.00088592
 1.89e-5

In [65]:
fit = single_element_fitness(p,0x0069,0.02);F = Diagonal(fit);  fit[0x0069:0x0069+3] # Fitness with 1000-fold advantage for 0x6

4-element Vector{Float64}:
 0.02
 1.0
 0.02
 0.02

In [48]:
F = Diagonal(fit);

In [66]:
normalize(F*E*P)[0x0069:0x0070]

8-element Vector{Float64}:
 1.4199986084013638e-6
 9.999990200009603e-7
 0.00014447985840973875
 1.2199988044011716e-6
 5.2739948314850655e-5
 2.9999970600028813e-7
 9.8919903058495e-5
 6.139993982805898e-6

In [122]:
M = mutation(map(Float64,E), 50.)  # 

256×256 Matrix{Float64}:
 9.44887   0.046321  0.044614  0.048461  …  0.004705  0.007345  0.415301
 0.046277  1.19138   0.004455  0.018294     0.00203   0.01248   0.007442
 0.044158  0.004464  1.12451   0.010939     0.007999  0.001904  0.004747
 0.048454  0.018144  0.010842  1.31483      0.001101  0.000938  0.023595
 0.034146  0.003922  0.000916  0.000175     0.000176  0.001252  0.003315
 0.06948   0.019434  0.000262  0.008524  …  0.000668  0.000679  0.027708
 0.000205  2.1e-5    0.000189  1.9e-5       1.8e-5    5.7e-5    4.8e-5
 0.0034    0.000797  0.000562  0.003085     0.000471  0.000144  0.00212
 0.030183  0.000636  0.001251  0.000218     0.000143  4.7e-5    0.002531
 1.2e-5    4.0e-5    2.0e-6    2.0e-6       4.0e-6    2.0e-6    6.0e-6
 0.046617  0.000158  0.006296  0.000499  …  7.9e-5    0.000105  0.009694
 0.000412  0.000312  0.000115  0.000703     1.8e-5    8.5e-5    0.000242
 0.024228  8.7e-5    0.000121  0.002647     9.7e-5    7.4e-5    0.005435
 ⋮                             

In [93]:
vc = vcat(collect(1:8),collect(0x0069+1:0x0070))

15-element Vector{Int64}:
   1
   2
   3
   4
   5
   6
   7
   8
 106
 107
 108
 109
 110
 111
 112

In [107]:
vlabels = map(x->[x],vc)

15-element Vector{Vector{Int64}}:
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]
 [7]
 [8]
 [106]
 [107]
 [108]
 [109]
 [110]
 [111]
 [112]

In [123]:
display_pheno_vect((M*P)[vc],vlabels)

,phenos,values
,Array…,Float64
1,[1],0.0462325
2,[2],0.00538966
3,[3],0.00490383
4,[4],0.00611535
5,[5],0.00462672
6,[6],0.00674898
7,[7],0.00391508
8,[8],0.00413473
9,[106],0.00390625


In [119]:
fit = single_element_fitness(p,0x0069,0.03);F = Diagonal(fit);  fit[0x0069:0x0069+3]

4-element Vector{Float64}:
 0.03
 1.0
 0.03
 0.03

In [124]:
display_pheno_vect(normalize((F*M)*P)[vc],vlabels)

,phenos,values
,Array…,Float64
1,[1],0.0349818
2,[2],0.00407809
3,[3],0.00371048
4,[4],0.00462718
5,[5],0.00350081
6,[6],0.00510662
7,[7],0.00296235
8,[8],0.00312855
9,[106],0.0985223


In [121]:
display_pheno_vect(normalize((F*M)^3*P)[vc],vlabels)

,phenos,values
,Array…,Float64
1,[1],0.141099
2,[2],0.000975565
3,[3],0.000848133
4,[4],0.00133143
5,[5],0.000646239
6,[6],0.00184951
7,[7],2.16038e-5
8,[8],0.00011617
9,[106],0.639144


In [104]:
EV= eigen(F*M); display_pheno_vect( -EV.vectors[:,end], labels )  # The 0x6 component is slightly larger than the 0x5 component
# This shows that the bias of the G-P map has been overcome

,phenos,values
,Array…,Float64
1,[0x00],0.174283
2,[0x01],0.090166
3,[0x02],0.0442081
4,[0x03],0.0185375
5,[0x04],0.0442081
6,[0x05],0.520246
7,[0x06],0.597832
8,[0x07],0.0908115
9,[0x08],0.090166


In [159]:
display_pheno_vect( normalize((F*M)^5*fill(1/16,16)), labels )  # background fitness of 0.85 gives a modest superiority of 0x6

,phenos,values
,Array…,Float64
1,[0x00],0.069739
2,[0x01],0.0596338
3,[0x02],0.0520971
4,[0x03],0.0517484
5,[0x04],0.0520971
6,[0x05],0.0847301
7,[0x06],0.0973664
8,[0x07],0.0596449
9,[0x08],0.0596338


In [158]:
display_pheno_vect( mx, labels ) 


,phenos,values
,Array…,Float64
1,[0x00],0.069739
2,[0x01],0.0596338
3,[0x02],0.0520971
4,[0x03],0.0517484
5,[0x04],0.0520971
6,[0x05],0.0847301
7,[0x06],0.0973664
8,[0x07],0.0596449
9,[0x08],0.0596338
